In [9]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [10]:
dropColumns=["tbl","citymarketid_1","citymarketid_2","airportid_1","airportid_2","tbl1apk"]
data=pd.read_csv('US Airline Flight Routes and Fares 1993-2024.csv').drop(columns=dropColumns).dropna().reset_index(drop=True)
data=data.loc[(data["city1"]!="Key West, FL") & (data["city2"]!="Key West, FL") & (data["city1"]!="Tulsa, OK") & (data["city2"]!="Tulsa, OK"),:].reset_index(drop=True)
# data=data.loc[(data["Year"]==year) & (data["quarter"]==quarter),:].reset_index(drop=True)


G = nx.MultiDiGraph()
GGen=nx.Graph()

dataNp=data["city1"].to_numpy()
posNp=data["Geocoded_City1"].to_numpy()
for airport, pos in zip(dataNp,posNp):
    pos=pos.split("(")[-1][:-1]
    latitude,longtitude=pos.split(",")
    pos=(float(latitude),float(longtitude))
    G.add_node(airport,pos=pos)
    G.add_node(airport,pos=pos)


dataNp=data["city2"].to_numpy()
posNp=data["Geocoded_City2"].to_numpy()
for airport, pos in zip(dataNp,posNp):
    pos=pos.split("(")[-1][:-1]
    latitude,longtitude=pos.split(",")
    pos=(float(latitude),float(longtitude))
    G.add_node(airport,pos=pos)
    GGen.add_node(airport,pos=pos)


dataNp=None
posNp=None

# flights=[]

dfNp=data.to_numpy()

for flight in dfNp:
    origin, dest, distance, fare, originPos, destPos = flight[2], flight[3], flight[6], flight[8], flight[-2], flight[-1]
    # flights.append((origin,dest,distance,fare,originPos,destPos))
    G.add_edge(origin,dest,weight=distance,price=fare)
    GGen.add_edge(origin,dest,weight=distance,price=fare)



C:\Users\alt_user\AppData\Local\Temp\ipykernel_2836\2817888692.py:2: DtypeWarning: Columns (20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv('US Airline Flight Routes and Fares 1993-2024.csv').drop(columns=dropColumns).dropna().reset_index(drop=True)


In [11]:
import time


from Functions import *
from FunctionsGenetic import *


def geneticAlg(G, start, end, sampleSize=100, parentCount=10, generations=100, loop=0):
    if(loop>3):
        return -1
    try:
        route = genetic_algorithm(GGen, start, end, sampleSize, parentCount, generations)
    except:
        return geneticAlg(G, start, end, sampleSize=100, parentCount=10, generations=100, loop=loop+1)
    return route[-1][-1]


genetic, astar, dijkstra_s, astar_multiDis, astar_multiMoney=(0,0,0,0,0)
geneticT, astarT, dijkstra_sT, astar_multiDisT = (0,0,0,0)
for i in range(100):
    start= data.sample(1)["city1"].values[0]
    end= data.sample(1)["city2"].values[0]
    
    start_time = time.time()
    distances, preds = dijkstra(G, start)
    if(distances[end]!=float('inf')):
        dijkstra_s+=distances[end]
    else:
        continue
    dijkstra_sT+=time.time() - start_time

    start_time = time.time()
    geneticTemp=geneticAlg(G, start, end, sampleSize=100, parentCount=10, generations=10)
    if(geneticTemp==-1):
        genetic+=distances[end]
    else:
        genetic+=geneticTemp
    geneticT+=time.time() - start_time

    start_time = time.time()
    distances, preds = a_star(G, start, end, heuristic)
    astar+=distances[end]
    astarT+=time.time() - start_time


    start_time = time.time()
    pareto_front, predecessors = multi_criteria_a_star(G, start, end, heuristic)
    astar_multiDisT+=time.time() - start_time
    paths = reconstruct_path(start, end, pareto_front, predecessors)
    
    
    for path, (distance, money) in paths:
        astar_multiMoney+=money
    print("Loop: ",i)


Loop:  1
Loop:  2
Loop:  3
Loop:  4
Loop:  5
Loop:  6
Loop:  8
Loop:  9
Loop:  10
Loop:  11
Loop:  12
Loop:  13
Loop:  14
Loop:  15
Loop:  16
Loop:  17
Loop:  18
Loop:  19
Loop:  21
Loop:  22
Loop:  23
Loop:  24
Loop:  25
Loop:  26
Loop:  27
Loop:  28
Loop:  29
Loop:  30
Loop:  31
Loop:  32
Loop:  33
Loop:  34
Loop:  36
Loop:  37
Loop:  38
Loop:  39
Loop:  40
Loop:  41
Loop:  42
Loop:  43
Loop:  44
Loop:  45
Loop:  46
Loop:  47
Loop:  48
Loop:  49
Loop:  51
Loop:  52
Loop:  53
Loop:  54
Loop:  55
Loop:  56
Loop:  57
Loop:  58
Loop:  59
Loop:  60
Loop:  61
Loop:  62
Loop:  63
Loop:  64
Loop:  67
Loop:  68
Loop:  69
Loop:  71
Loop:  72
Loop:  73
Loop:  74
Loop:  75
Loop:  76
Loop:  77
Loop:  78
Loop:  79
Loop:  80
Loop:  81
Loop:  83
Loop:  84
Loop:  85
Loop:  86
Loop:  87
Loop:  88
Loop:  90
Loop:  91
Loop:  92
Loop:  93
Loop:  94
Loop:  95
Loop:  96
Loop:  98


In [12]:
print("Genetic Algorithm: ", genetic, "Time: ", geneticT)
print("A* Algorithm: ", astar, "Time: ", astarT)
print("Dijkstra Algorithm: ", dijkstra_s, "Time: ", dijkstra_sT)
print("A* Multi Criteria Distance: (multiple paths)" , "Time: ", astar_multiDisT, "Money: ", astar_multiMoney)

Genetic Algorithm:  115522.0 Time:  146.72145128250122
A* Algorithm:  110140.0 Time:  0.9622185230255127
Dijkstra Algorithm:  109456.0 Time:  6.655618667602539
A* Multi Criteria Distance: (multiple paths) Time:  10.890886068344116 Money:  20928.290000000005
